# THE NEW KEYNESIAN MODEL

Imports and set magics:

In [30]:
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from ipywidgets import interact, FloatSlider
import sympy as sp
from scipy.optimize import minimize


# Model description

A New Keynesian model is a framework used in macroeconomics to analyze the effects of various economic shocks on key macroeconomic variables such as output, inflation, and interest rates. It combines elements of Keynesian economics with modern macroeconomic theory. A typical New Keynesian model consists of three main equations:

IS Curve:
$$
y_t = E_t[y_{t+1}] - 1/σ * (E_t[r_t] - ρ) 
$$
Phillips Curve:
$$
π_t = κy_t + βE_t[π_{t+1}]
$$
Taylor Rule:
$$
r_t = ρ + ϕ_yE_t[y_{t+1}] + ϕ_πE_t[π_{t+1}] 
$$

The IS Curve represents the aggregate demand equation which shows the relationship between output, expected future output, the real interest rate. It reflects how changes in interest rates and expectations about future output affect current output. The Phillips Curve is the inflation equation, which represents the relationship between inflation, the output gap and eexpected future inflation. It describes how deviations of actual output from potential output influence inflation. Taylor Rule or the Monetary Policy Rule represents the central bank's reaction function, where the nominal interest rate is set based on deviations of output from potential output and inflation from target inflation.

Firstly, the three equations of the model have been coded.

Next, the model has been solved using fsolve, which is an optimization function from scipy.optimize.The initial values of output, inflaton and interest rate have been assumed to be 0. Then the model has been simulated.

The reason of using fsolve rather than defining y_t, pi_t and r_t directly from the three equations is that we are dealing with a system of nonlinear equations that are interdependent. The Taylor rule, Phillips curve, and IS curve must all hold. It would not be sufficient to define these variables straight from the equations to guarantee simultaneous satisfaction of the three requirements. 

This approach ensures that the interdependencies and nonlinearities are properly accounted for, providing a consistent solution for the model's variables.


Instead of simply displaying the values of each variable, we will plot the simulated paths for key macroeconomic variables in response to different shocks. This provides a more intuitive understanding of how the economy reacts over time. The plots will include:
- Output Gap
- Inflation Rate
- Nominal Interest Rate
- Comparison to steady state values

Lastly, an interactive plot has been created. 

(If after running all the project the grpah does not appear, run the cell alone)

In [35]:
# Defining the model equations:
# IS curve
def y_t(Exp_y_t1, sigma, Exp_r_t, rho):
    return Exp_y_t1 - (1/sigma)*(Exp_r_t-rho)

# Phillips curve
def pi_t(kappa, y_t, beta, Exp_pi_t1):
    return kappa*y_t + beta* Exp_pi_t1

# Taylor rule
def r_t(rho, phi_y, phi_pi, Exp_y_t1, Exp_pi_t1):
    return rho + phi_y * Exp_y_t1 + phi_pi * Exp_pi_t1

# Solving the model equations
def solve_model(Exp_y_t1, Exp_r_t, Exp_pi_t1, sigma, rho, kappa, beta, phi_y, phi_pi):
    # Define a function to solve using fsolve
    def equations(vars):
        yt, pi_t1, rt1 = vars
        eq1 = y_t(Exp_y_t1, sigma, rt1, rho) - yt
        eq2 = pi_t(kappa, yt, beta, Exp_pi_t1) - pi_t1
        eq3 = r_t(rho, phi_y, phi_pi, Exp_y_t1, Exp_pi_t1) - rt1
        return [eq1, eq2, eq3]
    
    # Initial guess
    guess = [0, 0, 0]
    
    # Solve using fsolve
    yt, pi_t1, rt1 = fsolve(equations, guess)
    
    return yt, pi_t1, rt1

# Simulation
def simulate_model(periods, Exp_y_t1, Exp_r_t, Exp_pi_t1, sigma, rho, kappa, beta, phi_y, phi_pi):
    y_t_values = [0]  # Initial output assumed to be 0
    pi_t_values = [0]  # Initial inflation assumed to be 0
    r_t_values = [0]  # Initial interest rate assumed to be 0
    for _ in range(int(periods)):  # Convert periods to integer
        yt, pi_t1, rt1 = solve_model(Exp_y_t1, Exp_r_t, Exp_pi_t1, sigma, rho, kappa, beta, phi_y, phi_pi)
        y_t_values.append(yt)
        pi_t_values.append(pi_t1)
        r_t_values.append(rt1)
        # Update expected values for the next period
        Exp_y_t1 = yt
        Exp_r_t = rt1
        Exp_pi_t1 = pi_t1
    return y_t_values, pi_t_values, r_t_values

# Interactive plot function
def interactive_plot(periods, Exp_y_t1_val, Exp_r_t_val, Exp_pi_t1_val, sigma_val, rho_val, kappa_val, beta_val, phi_y_val, phi_pi_val):
    y_t_values, pi_t_values, r_t_values = simulate_model(periods, Exp_y_t1_val, Exp_r_t_val, Exp_pi_t1_val, sigma_val, rho_val, kappa_val, beta_val, phi_y_val, phi_pi_val)
    
    plt.figure(figsize=(10, 6))

    plt.subplot(3, 1, 1)
    plt.plot(y_t_values)
    plt.title('Output (y_t)')
    plt.xlabel('Period')
    plt.ylabel('Output')

    plt.subplot(3, 1, 2)
    plt.plot(pi_t_values)
    plt.title('Inflation (π_t)')
    plt.xlabel('Period')
    plt.ylabel('Inflation')

    plt.subplot(3, 1, 3)
    plt.plot(r_t_values)
    plt.title('Interest Rate (r_t)')
    plt.xlabel('Period')
    plt.ylabel('Interest Rate')

    plt.tight_layout()
    plt.show()

# Set up the interactive widget
interact(interactive_plot,
         periods=FloatSlider(min=1, max=100, step=1, value=100, description='Periods'),
         Exp_y_t1_val=FloatSlider(min=0, max=1, step=0.1, value=0.5, description='Exp_y_t1'),
         Exp_r_t_val=FloatSlider(min=0, max=1, step=0.1, value=0.5, description='Exp_r_t'),
         Exp_pi_t1_val=FloatSlider(min=0, max=1, step=0.1, value=0.5, description='Exp_pi_t1'),
         sigma_val=FloatSlider(min=0, max=2, step=0.1, value=1.5, description='sigma'),
         rho_val=FloatSlider(min=0, max=1, step=0.1, value=0.5, description='rho'),
         kappa_val=FloatSlider(min=0, max=1, step=0.1, value=0.5, description='kappa'),
         beta_val=FloatSlider(min=0, max=1, step=0.1, value=0.99, description='beta'),
         phi_y_val=FloatSlider(min=0, max=1, step=0.1, value=0.5, description='phi_y'),
         phi_pi_val=FloatSlider(min=0, max=2, step=0.1, value=1.5, description='phi_pi')
        )

interactive(children=(FloatSlider(value=100.0, description='Periods', min=1.0, step=1.0), FloatSlider(value=0.…

<function __main__.interactive_plot(periods, Exp_y_t1_val, Exp_r_t_val, Exp_pi_t1_val, sigma_val, rho_val, kappa_val, beta_val, phi_y_val, phi_pi_val)>

## Analytical solution

From the values ploted before, we will solve the model by choosing random values.

Here, we wanted first to solve the model as it is, so we wrote some code in order to find the optimal values for each variables we have defined above, except for "periods" which will be set as 100. 

We wanted to set all the variable as "completely" random so that we could find the steady state of the model. 

In [32]:
# Define symbols
y_t, y_t_1, pi_t, pi_t_1, r_t, rho, sigma, kappa, beta, phi_y, phi_pi = sp.symbols('y_t y_t_1 pi_t pi_t_1 r_t rho sigma kappa beta phi_y phi_pi')

# Define equations
is_curve = sp.Eq(y_t, y_t_1 - (1/sigma) * (r_t - rho))
phillips_curve = sp.Eq(pi_t, kappa * y_t + beta * pi_t_1)
taylor_rule = sp.Eq(r_t, rho + phi_y * y_t_1 + phi_pi * pi_t_1)

# Solve the system of equations
solution = sp.solve((is_curve, phillips_curve, taylor_rule), (y_t, pi_t, r_t))

# Extract solutions
y_t_solution = solution[y_t]
pi_t_solution = solution[pi_t]
r_t_solution = solution[r_t]

# Print solutions
print("Output (y_t) = ", y_t_solution)
print("Inflation (pi_t) = ", pi_t_solution)
print("Nominal Interest Rate (r_t) = ", r_t_solution)

Output (y_t) =  (-phi_pi*pi_t_1 - phi_y*y_t_1 + sigma*y_t_1)/sigma
Inflation (pi_t) =  (beta*pi_t_1*sigma - kappa*phi_pi*pi_t_1 - kappa*phi_y*y_t_1 + kappa*sigma*y_t_1)/sigma
Nominal Interest Rate (r_t) =  phi_pi*pi_t_1 + phi_y*y_t_1 + rho


Here I only rewrote a nicer way the result stated above.

$$
y_{t} = (-ϕ_π * π_{t+1} - ϕ_y * y_{t+1} + σ * y_{t+1})/σ 
$$

$$
π_{t} = (β * π_{t+1} * σ - κ * ϕ_π * π_{t+1} - κ * ϕ_y * y_{t+1} + κ * σ * y_{t+1})/σ
$$

$$
r_{t} = ϕ_π * π_{t+1} + ϕ_y * y_{t+1} + ρ
$$

In [33]:
# Define symbols
y, pi, r, rho, sigma, kappa, beta, phi_y, phi_pi = sp.symbols('y pi r rho sigma kappa beta phi_y phi_pi')

# Define equations
is_curve = sp.Eq(y, y - (1/sigma) * (r - rho))
phillips_curve = sp.Eq(pi, kappa * y + beta * pi)
taylor_rule = sp.Eq(r, rho + phi_y * y + phi_pi * pi)

# Solve the system of equations for steady state
steady_state_solution = sp.solve((is_curve, phillips_curve, taylor_rule), (y, pi, r))

# Print steady-state solutions
print("Steady-State Output (y) = ", steady_state_solution[y])
print("Steady-State Inflation (pi) = ", steady_state_solution[pi])
print("Steady-State Nominal Interest Rate (r) = ", steady_state_solution[r])

Steady-State Output (y) =  0
Steady-State Inflation (pi) =  0
Steady-State Nominal Interest Rate (r) =  rho


Here we tried to find the equilibrium values, since we did not define any constant we gave the values with letters.

But we wrote a code that should give us the steady-state for the output, inflation and interest rate.
The first two one should be equal to 0 which is consistent with what we find in the interactive plot in the first part of the assignment on the long term. For the last variable we see that it should be equal to $\rho$ which is again consistent with the values we have in the first part of the assignment.

## Numerical solution

In here we are adding some shocks to our equations. We wanted to see how these shocks would affect the economy so we simulate them into a interactive plot. Like this we can see the impact of each shocks, for positive and negative values on a short and long run. 

For example, if the u_cp = -0.60 then the output will increase whereas if u_MP = 0.80 the output will decrease.


IS Curve:
$$
y_t = E_t[y_{t+1}] - 1/σ * (E_t[r_t] - ρ) + u_{IS}
$$
Phillips Curve:
$$
π_t = κy_t + βE_t[π_{t+1}] + u_{CP}
$$
Taylor Rule:
$$
r_t = ρ + ϕ_yE_t[y_{t+1}] + ϕ_πE_t[π_{t+1}] + u_{MP}
$$

(If after running all the project the graph does not appear, run the cell independently)

In [36]:
# Defining the model equations with exogenous shocks
def y_t(Exp_y_t1, sigma, Exp_r_t, rho, uIS):
    return Exp_y_t1 - (1/sigma)*(Exp_r_t - rho) + uIS

def pi_t(kappa, y_t, beta, Exp_pi_t1, uCP):
    return kappa * y_t + beta * Exp_pi_t1 + uCP

def r_t(rho, phi_y, phi_pi, Exp_y_t1, Exp_pi_t1, uMP):
    return rho + phi_y * Exp_y_t1 + phi_pi * Exp_pi_t1 + uMP

# Solving the model equations with exogenous shocks
def solve_model(Exp_y_t1, Exp_r_t, Exp_pi_t1, sigma, rho, kappa, beta, phi_y, phi_pi, uIS, uCP, uMP):
    def equations(vars):
        yt, pi_t1, rt1 = vars
        eq1 = y_t(Exp_y_t1, sigma, rt1, rho, uIS) - yt
        eq2 = pi_t(kappa, yt, beta, Exp_pi_t1, uCP) - pi_t1
        eq3 = r_t(rho, phi_y, phi_pi, Exp_y_t1, Exp_pi_t1, uMP) - rt1
        return [eq1, eq2, eq3]
    
    guess = [0, 0, 0]
    yt, pi_t1, rt1 = fsolve(equations, guess)
    
    return yt, pi_t1, rt1

# Simulation with exogenous shocks
def simulate_model(periods, Exp_y_t1, Exp_r_t, Exp_pi_t1, sigma, rho, kappa, beta, phi_y, phi_pi, uIS, uCP, uMP):
    y_t_values = [0]
    pi_t_values = [0]
    r_t_values = [0]
    for _ in range(int(periods)):
        yt, pi_t1, rt1 = solve_model(Exp_y_t1, Exp_r_t, Exp_pi_t1, sigma, rho, kappa, beta, phi_y, phi_pi, uIS, uCP, uMP)
        y_t_values.append(yt)
        pi_t_values.append(pi_t1)
        r_t_values.append(rt1)
        # Update expected values for the next period
        Exp_y_t1 = yt
        Exp_r_t = rt1
        Exp_pi_t1 = pi_t1
    return y_t_values, pi_t_values, r_t_values

# Interactive plot function
def interactive_plot(periods, Exp_y_t1_val, Exp_r_t_val, Exp_pi_t1_val, sigma_val, rho_val, kappa_val, beta_val, phi_y_val, phi_pi_val, uIS_val, uCP_val, uMP_val):
    y_t_values, pi_t_values, r_t_values = simulate_model(periods, Exp_y_t1_val, Exp_r_t_val, Exp_pi_t1_val, sigma_val, rho_val, kappa_val, beta_val, phi_y_val, phi_pi_val, uIS_val, uCP_val, uMP_val)
    
    plt.figure(figsize=(12, 10))

    plt.subplot(4, 1, 1)
    plt.plot(y_t_values)
    plt.title('Output (y_t)')
    plt.xlabel('Period')
    plt.ylabel('Output')

    plt.subplot(4, 1, 2)
    plt.plot(pi_t_values)
    plt.title('Inflation (π_t)')
    plt.xlabel('Period')
    plt.ylabel('Inflation')

    plt.subplot(4, 1, 3)
    plt.plot(r_t_values)
    plt.title('Interest Rate (r_t)')
    plt.xlabel('Period')
    plt.ylabel('Interest Rate')

    plt.subplot(4, 1, 4)
    plt.plot(uIS_val * np.ones(len(y_t_values)), label='uIS')
    plt.plot(uCP_val * np.ones(len(pi_t_values)), label='uCP')
    plt.plot(uMP_val * np.ones(len(r_t_values)), label='uMP')
    plt.title('Exogenous Shocks')
    plt.xlabel('Period')
    plt.ylabel('Shocks')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Set up the interactive widget
interact(interactive_plot,
         periods=FloatSlider(min=1, max=100, step=1, value=100, description='Periods'),
         Exp_y_t1_val=FloatSlider(min=0, max=1, step=0.1, value=0.5, description='Exp_y_t1'),
         Exp_r_t_val=FloatSlider(min=0, max=1, step=0.1, value=0.5, description='Exp_r_t'),
         Exp_pi_t1_val=FloatSlider(min=0, max=1, step=0.1, value=0.5, description='Exp_pi_t1'),
         sigma_val=FloatSlider(min=0, max=2, step=0.1, value=1.5, description='sigma'),
         rho_val=FloatSlider(min=0, max=1, step=0.1, value=0.5, description='rho'),
         kappa_val=FloatSlider(min=0, max=1, step=0.1, value=0.5, description='kappa'),
         beta_val=FloatSlider(min=0, max=1, step=0.1, value=0.99, description='beta'),
         phi_y_val=FloatSlider(min=0, max=1, step=0.1, value=0.5, description='phi_y'),
         phi_pi_val=FloatSlider(min=0, max=2, step=0.1, value=1.5, description='phi_pi'),
         uIS_val=FloatSlider(min=-1, max=1, step=0.1, value=0, description='uIS'),
         uCP_val=FloatSlider(min=-1, max=1, step=0.1, value=0, description='uCP'),
         uMP_val=FloatSlider(min=-1, max=1, step=0.1, value=0, description='uMP')
        )

interactive(children=(FloatSlider(value=100.0, description='Periods', min=1.0, step=1.0), FloatSlider(value=0.…

<function __main__.interactive_plot(periods, Exp_y_t1_val, Exp_r_t_val, Exp_pi_t1_val, sigma_val, rho_val, kappa_val, beta_val, phi_y_val, phi_pi_val, uIS_val, uCP_val, uMP_val)>

# Conclusion

The purpose of the New Keynesian model is to analyze how various economic shocks (demand shock, cost-push shock, monetary policy shock) affect key macroeconomic variables such as output, inflation, and interest rates. It helps economists and policymakers understand the dynamics of the economy and design appropriate policy responses to stabilize the economy, such as monetary policy adjustments in response to changes in output and inflation. 
The model could be extended by a fourth equation known as the Output Gap Evolution Equation.It describes how potential output evolves over time and depends on exogenous factors like technological progress or demographic changes.

Output Gap Evolution Equation:
$$
Y^*_{t+1} = Y^*_{t} +γ(Y_t - Y^*_{t+}) + ϵ
$$
The purpose of including this equation is to capture the long-term trends in the economy and how they influence short-term dynamics. 
​